In [165]:
import message_db
import sqlite3
import re

### Connect Config

In [167]:
con, cur = message_db.connect()

In [3]:
cur.row_factory

<function message_db.dict_factory>

In [166]:
con.close()

### Query Table Counts

In [67]:
tables = ['chat', 'message', 'handle']

count_rows = lambda table_name: cur.execute('SELECT COUNT(*) AS count FROM {}'.format(table_name)).fetchone()['count']
table_counts = {table: count_rows(table) for table in tables}

In [68]:
table_counts

{'chat': 743, 'handle': 654, 'message': 248618}

### Reading SQL query file

In [ ]:
fd = open('messages.sql', 'r')
sql_file = fd.read()
fd.close()

In [ ]:
sql_commands = [re.sub('\n\s{0}', ' ', cmd) for cmd in sql_file.split(';') if cmd is not '']

### Chat Table Queries

SELECTs all chats and builds a dictionary of chat_id --> chat_info

In [88]:
chat_query = 'SELECT ROWID AS id, account_id, service_name AS service FROM chat'

In [89]:
cur.execute(chat_query)
chat_result = cur.fetchall()

In [90]:
chats = {c['id']: c for c in chat_result}

### Handle Table

Selects all handles builds a dictionary mapping handle_ids to handle info (phone number)

In [84]:
handle_query = 'SELECT ROWID AS id, id AS full_number, country, service, uncanonicalized_id AS local_number, person_centric_id AS person_id FROM handle'

In [85]:
cur.execute(handle_query)
handle_result = cur.fetchall()

In [86]:
handles = {h['id']: h for h in handle_result}

### Messages Table

Selects all messages and builds a list to be used for bucketing of messages to their corresponding chats / handles

In [101]:
msg_query = "SELECT ROWID AS id, text, is_from_me, datetime((date/1000000000) + strftime('%s', '2001-01-01 00:00:00'), 'unixepoch', 'localtime') AS date FROM message ORDER BY date DESC"

In [102]:
cur.execute(msg_query)
message_result = cur.fetchall()

In [103]:
messages = {m['id']: m for m in message_result}

In [104]:
messages['113849']

{'date': '2017-03-10 23:56:13',
 'id': 113849,
 'is_from_me': 0,
 'text': "I'm their only hope"}

### Chat - Message Join Table

In [46]:
cmj_query = "SELECT cmj.message_id AS message_id, cmj.chat_id AS chat_id, text, is_from_me, handle_id, datetime((date/1000000000) + strftime('%s', '2001-01-01 00:00:00'), 'unixepoch', 'localtime') AS date FROM chat_message_join AS cmj INNER JOIN message AS m ON cmj.message_id = m.ROWID ORDER BY date"

In [47]:
cur.execute(cmj_query)
cmj_result = cur.fetchall()

### Chat - Handle Join Table

In [181]:
chj_query = "SELECT chj.handle_id AS handle_id, chj.chat_id AS chat_id, id AS full_number, country, service, uncanonicalized_id AS local_number, person_centric_id AS person_id FROM chat_handle_join AS chj INNER JOIN handle AS h ON chj.handle_id = h.ROWID ORDER BY handle_id"

In [51]:
cur.execute(chj_query)
chj_result = cur.fetchall()

### Chat - Handle - Message Query

#### NOTE
1. some messages have duplicates as iMessage/SMS in chat_message_join
2. some messages (presumably attachments) have NULL text values

In [179]:
combined_query = "SELECT m.ROWID AS message_id, cmj.chat_id AS chat_id, m.handle_id AS handle_id, text, is_from_me, datetime((date/1000000000) + strftime('%s', '2001-01-01 00:00:00'), 'unixepoch', 'localtime') AS date FROM message AS m INNER JOIN chat_message_join AS cmj ON cmj.message_id = m.ROWID LEFT JOIN chat_handle_join AS chj ON cmj.chat_id = chj.handle_id WHERE text IS NOT NULL GROUP BY message_id"
cur.execute(combined_query)
full_messages = cur.fetchall()